### Money Manager Ex

In [ ]:
import pandas as pd

def convert_axis_to_mmex(input_csv_path, output_csv_path):
    """
    Converts an Axis Bank statement CSV to a Money Manager Ex importable CSV file.

    Args:
        input_csv_path (str): The file path of the Axis Bank statement CSV.
        output_csv_path (str): The file path to save the converted Money Manager Ex CSV.
    """
    try:
        # Read the Axis Bank CSV file. You might need to adjust skiprows if your
        # CSV has some header rows before the actual transaction data.
        axis_df = pd.read_csv(input_csv_path, skiprows=14) # Adjust skiprows as needed

        # Rename columns for easier access (optional, but good practice)
        axis_df.columns = [
            'Sr. No.', 'Transaction Date', 'Value Date', 'Description',
            'Cheque Number', 'Debit', 'Credit', 'Balance', 'Branch Name'
        ]

        # Create a new DataFrame for Money Manager Ex format
        mmex_df = pd.DataFrame(columns=[
            'Date', 'Payee', 'Transaction Type', 'Amount',
            'Category', 'SubCategory', 'Notes'
        ])

        # Process each transaction from the Axis Bank statement
        for index, row in axis_df.iterrows():
            date = row['Transaction Date']
            payee = row['Description']
            notes = '' # You can populate this if you have relevant info

            # Determine if it's a deposit or withdrawal
            if pd.notna(row['Credit']) and row['Credit'] > 0:
                transaction_type = 'Deposit'
                amount = row['Credit']
            elif pd.notna(row['Debit']) and row['Debit'] > 0:
                transaction_type = 'Withdrawal'
                amount = row['Debit']
            else:
                continue  # Skip rows that are not transactions

            # Add the processed row to the new DataFrame
            new_row = pd.DataFrame([{
                'Date': date,
                'Payee': payee,
                'Transaction Type': transaction_type,
                'Amount': amount,
                'Category': '',  # Leave category blank to be filled in MMEX
                'SubCategory': '',
                'Notes': notes
            }])
            mmex_df = pd.concat([mmex_df, new_row], ignore_index=True)

        # Save the formatted DataFrame to a new CSV file
        # index=False prevents writing row numbers to the file [4, 5]
        mmex_df.to_csv(output_csv_path, index=False)
        print(f"Successfully converted '{input_csv_path}' to '{output_csv_path}' for Money Manager Ex.")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- How to use the script ---

# 1. Replace 'path/to/your/axis_statement.csv' with the actual path to your downloaded statement.
#    You can download your statement in CSV format from Axis Bank's internet banking portal. [17]
input_file = 'path/to/your/axis_statement.csv'

# 2. Replace 'path/to/your/mmex_import.csv' with the desired path for the output file.
output_file = 'path/to/your/mmex_import.csv'

# 3. Run the conversion function.
convert_axis_to_mmex(input_file, output_file)